 ### Data << Credit_Risk_Model_Gaussian_1Factor_Copula.zip

In [1]:
cd C:\Users\USER\수정용_금융리스크관리

C:\Users\USER\수정용_금융리스크관리


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns

### Credit Risk Capital using 1 Factor Gaussian Copula Simulation

$$
\begin{array}{c}
Z =\beta M + \sqrt{1-\beta^2} \bullet \epsilon \\ \ \ \ \  = \sqrt{\rho} M + \sqrt{1-\rho} \bullet \epsilon \\ \\ U = N(Z), \ \tau = \frac{-ln(1-U)}{PD} \\
\end{array}
$$

$$
Risk \ Capital = VaR_{0.999} - E(Loss)
$$

 ## Default Correlation
 
 $$\begin{array}{c}
 \rho_{(A,B)} = \frac{P(A and B) - P(A)P(B)}{\sqrt{P(A)(1-P(A))}\sqrt{P(B)(1-P(B))} } \\ \\
 \bar{\rho}_{Default} = \frac{Joint - PD^2}{PD(1-PD)}  \\ \\ 
 Joint = \frac{X(X-1)}{Y(Y-1)} \\  X = number \ of \ default \\ Y = number \ of \ loan
 \end{array}$$

In [3]:
Z_fun = lambda copula_corr , M , epsilon : np.sqrt(copula_corr) * M + np.sqrt(1-copula_corr) * epsilon
tau_fun = lambda Z,PD : -np.log(1-norm.cdf(Z))/PD
def one_factor_copula_simulation(port_principal_list,copula_corr,T,RR,PD,simul_num = 10000,alpha= 0.001) :
    ###############################################
    # port_principal_list is Principal Value List #
    # for example : [100,100,100,...]             #
    ###############################################
    LGD = 1-RR
    Total_num = len(port_principal_list)
    M = np.random.normal(size = (1,simul_num))
    ##########################
    # fast simulation method #
    ##########################
    half_simul = int(simul_num/2+0.5)
    e1 = np.random.normal(size = (Total_num,half_simul))
    e2 = -e1
    epsilon = np.concatenate([e1,e2],axis=1)[:,:simul_num]
    Z = Z_fun(copula_corr, M, epsilon)#np.sqrt(copula_corr) * M + np.sqrt(1-copula_corr) * epsilon
    tau = tau_fun(Z,PD)
    Simul_Loss_Matrix = np.array(port_principal_list).reshape(-1,1) * (tau<T) * LGD
    Total_Simul_Loss = Simul_Loss_Matrix.sum(0)
    X = (Simul_Loss_Matrix>0).sum(0)
    Y = Total_num
    Joint_Prob = X*(X-1)/(Y*(Y-1))
    Default_Corr = ((Joint_Prob - PD**2)/(PD * (1-PD))).mean()
    Default_Rate = X/Y
    WCDR = pd.Series(X/Y).quantile(1-alpha)
    EDR = Default_Rate.mean()
    Default_Rate_Std = Default_Rate.std()
    EL = Total_Simul_Loss.mean()
    UEL = pd.Series(Total_Simul_Loss).quantile(1-alpha)
    RC = UEL - EL
    return {'simul_result' : Total_Simul_Loss ,'expected_loss' :EL, 'unexpected_loss':UEL, 'Risk_Capital':RC, 'Default_Corr' : Default_Corr, "EDR" : EDR, "Default_Rate_Std": Default_Rate_Std, 'WCDR' : WCDR}

def Calibrate_Copula_Corr(port_principal_list,actual_default_std, T, RR,PD, simul_num = 10000) :
    DR_std = np.vectorize(lambda i : one_factor_copula_simulation(port_principal_list,i,T,RR,PD,simul_num)['Default_Rate_Std'])
    Corr_Range = np.linspace(0,0.8,20+1)
    Simulated_Std = DR_std(Corr_Range)
    Copula_Correl = Corr_Range[np.abs(Simulated_Std - actual_default_std).argmin()]
    return Copula_Correl

In [4]:
one_factor_copula_simulation(port_principal_list = [8]*100+[20]*20,
                             copula_corr = 0.3,
                             T = 1,
                             RR = 0.5,
                             PD = 0.01,
                             simul_num = 10000,
                             alpha = 0.001)

{'simul_result': array([0., 0., 0., ..., 0., 4., 0.]),
 'expected_loss': 6.0168,
 'unexpected_loss': 152.0,
 'Risk_Capital': 145.9832,
 'Default_Corr': 0.0497297909062615,
 'EDR': 0.010015000000000001,
 'Default_Rate_Std': 0.02388612562183793,
 'WCDR': 0.25834166666666836}

In [5]:
Calibrate_Copula_Corr(port_principal_list=[100]*2000,
                      actual_default_std = 0.02, 
                      T = 1, 
                      RR = 0.5,
                      PD = 0.012, simul_num = 10000)

0.24

 ## Copula Analytic Calculation
 $$
\begin{array}{c}
WCDR(1yr,0.999, PD, \rho ) = N( \frac{ N^{-1}(PD) + \sqrt{\rho} N^{-1}(0.999) }
{\sqrt{1-\rho}   }  )\\\ Credit \ Risk \ Capital = VaR(0.999) - EL \\
\end{array}
$$

In [6]:
def Credit_VaR_Gaussian_Copula(port_principal_list, copula_corr, RR, PD, alpha = 0.001) :
    LGD = 1-RR
    Total_Principal = np.array(port_principal_list).sum()
    WCDR = norm.cdf((norm.ppf(PD) +np.sqrt(copula_corr) * norm.ppf(1-alpha))/(np.sqrt(1-copula_corr)) )
    UEL = Total_Principal * WCDR * LGD
    return {'unexpected_loss':UEL,'WCDR':WCDR}

def Credit_Risk_Capital_Gaussian_Copula(port_principal_list , copula_corr, RR , PD , alpha = 0.001) :
    Total_Principal = np.array(port_principal_list).sum()
    LGD = 1-RR
    my_dict = Credit_VaR_Gaussian_Copula(port_principal_list, copula_corr, RR, PD, alpha )
    P_range = np.arange(0.001,1,0.001)
    Default_Rate = norm.cdf((norm.ppf(PD) + np.sqrt(copula_corr) * norm.ppf(P_range))/np.sqrt(1-copula_corr))
    EDR = Default_Rate.mean()
    UEL = my_dict['unexpected_loss']
    EL = Total_Principal *EDR * (1-RR)
    RC = UEL - EL
    my_dict['expected_loss'] = EL , 
    my_dict['Risk_Capital'] = RC
    my_dict['EDR'] = EDR
    return my_dict

In [7]:
Credit_Risk_Capital_Gaussian_Copula(port_principal_list = [8]*100+[20]*20,
                                    copula_corr = 0.3,
                                    RR = 0.5,
                                    PD = 0.01,
                                    alpha = 0.001)

{'unexpected_loss': 134.6276948310477,
 'WCDR': 0.2243794913850795,
 'expected_loss': (5.905262285805912,),
 'Risk_Capital': 128.7224325452418,
 'EDR': 0.00984210380967652}

 ### Estimate Copula Correlation
 
 
 $$\begin{array}{c}
 Z =\beta M + \sqrt{1-\beta^2} \bullet \epsilon  \\  
 = \sqrt{\rho_{cop}} M + \sqrt{1-\rho_{cop}} \bullet \epsilon
 \\ \\
 Estimate \  \rho_{copula} \ with \  \rho_{equity}  \\ \\ 
 minimize \sum_{n=1}^{N} \sum_{m=1}^{M} {[{\rho_{equity}(n,m)} - \beta_n \beta_m]}^2 \ \ s.t. \ \ 0<=\beta_i <1 \
\\
 minimize \sum_{n=1}^{N} \sum_{m=1}^{M} {[{\rho_{equity}(n,m)} - \rho_{copula}]}^2 \ \ s.t. \ \ 0<=\rho_{copula} <1 \
\end{array}$$

In [8]:
def copula_correlation_with_equity(equity_price_data , start_day , end_day) :
    Price = pd.DataFrame(equity_price_data).resample('M').last()
    Return = Price.pct_change()[pd.to_datetime(start_day) : pd.to_datetime(end_day)]
    eq_corr = pd.DataFrame(np.triu(Return.corr())).applymap(lambda x : np.nan if x == 1 or x == 0 else x)
    eq_corr_ary = eq_corr.values.reshape(-1)
    corrs = pd.Series(eq_corr_ary)[pd.Series(eq_corr_ary).isna() == False].values.reshape(-1,1)
    ############################
    ## Copula Corr Straint 0 ~ 1 
    copul_corr_range = np.arange(0,1,0.02).reshape(1,-1)
    #################################
    ## (corr_equity - corr_copul)^2 #
    min_number = ((corrs - copul_corr_range)**2).sum(0).argmin()
    copul_corr_range.reshape(-1)[min_number]
    copul_corr = copul_corr_range.reshape(-1)[min_number]
    return copul_corr

In [9]:
Price = pd.read_csv('수정주가.csv',index_col = 0, parse_dates = ['Symbol']).resample('M').last()
equity_price_data = Price[Price.columns[:200]]
copula_correlation_with_equity(equity_price_data , start_day ='2000-01-31', end_day = '2020-12-31')

0.24